In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from datetime import timedelta
from datetime import datetime
import f_OTN
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [35]:
R = ['Meeting National <br> Est Lyonnais',
     'Meeting International <br> de Montreuil',
     'Meeting International <br> de Marseille',
     'Meeting Stanislas Nancy',
     'Meeting International <br> de Forbach',
     "Meeting International <br> de Troyes Aube",
    'Nationales Abendsportfest - Pfungstadt, TSV Stadion, Pfungstadt', 
    'ON Track Night Vienna <br> PB - minimas olympiques']

T = [timedelta(minutes=1, seconds=45, milliseconds=240),
     timedelta(minutes=1, seconds=46, milliseconds=760),
     timedelta(minutes=1, seconds=45, milliseconds=810),
     timedelta(minutes=1, seconds=45, milliseconds=220),
     'DNF',
     timedelta(minutes=1, seconds=46, milliseconds=940),
     timedelta(minutes=1, seconds=45, milliseconds=00),
     timedelta(minutes=1, seconds=44, milliseconds=250)]

D = [datetime(year=2024, month=5, day=4),
     datetime(year=2024, month=5, day=16),
     datetime(year=2024, month=5, day=22),
     datetime(year=2024, month=5, day=25),
     datetime(year=2024, month=5, day=26),
     datetime(year=2024, month=6, day=16),
     datetime(year=2024, month=6, day=19),
     datetime(year=2024, month=6, day=22)]

corentin = pd.DataFrame(f_OTN.remplir_dico_athle(R,T,D))

corentin['sec'] = corentin['Time'].apply(lambda x: 100 if x == 'DNF' else x.total_seconds())
corentin.sort_values(by='Date', inplace=True)

def format_race_date(row):
    if row['Time'] == 'DNF':
        return f"<b> DNF </b> <br> {row['Races']} ({row['Date'].strftime('%d/%m/%Y')})"
    else:
        return f"{row['Races']} <br> ({row['Date'].strftime('%d/%m/%Y')})"

corentin['Race_Date'] = corentin.apply(format_race_date, axis=1)
fig = px.line(corentin, x='Date', y='sec', color='Races', markers=True,
              range_x=[corentin['Date'].min(), corentin['Date'].max()],
              range_y=[corentin['sec'].min() - 5, corentin['sec'].max() + 5],
              title="Évolution des chronos sur 800m de Corentin LE CLEZIO (été 2024)",
              labels={'Date': 'Date de la course', 'sec': 'Chrono (en secondes)'},
              line_shape='spline',
              hover_name='Race_Date')


fig.update_layout(
    width=1000,  # Largeur du graphique
    height=700,  # Hauteur du graphique
    showlegend=True,
    legend=dict(
        traceorder='normal',
        font=dict(size=10),
        orientation="v",
        x=1,
        y=1
    )
)


frames = []


for i in range(len(corentin)):
    frame_data = corentin.iloc[:i+1]  
    frame_annotations = [
        dict(
            x=corentin.iloc[j]['Date'],
            y=corentin.iloc[j]['sec'],
            text=corentin.iloc[j]['Race_Date'],
            showarrow=True,
            arrowhead=1,
            xanchor='left',
            yanchor='bottom',  
            ax=20,  
            ay=-30
        )
        for j in range(i+1)
    ]
    trace = px.scatter(corentin.iloc[i:i+1], x='Date', y='sec', color='Races').data[0]
    trace.marker.size = 10  
    
    frames.append({
        'data': [fig.data[0], trace],  
        'layout': {'annotations': frame_annotations}
    })
fig.frames = frames
fig.update_layout(
    updatemenus=[{
        "buttons": [{
            "args": [None, {
                "frame": {"duration": 3000, "redraw": True},  
                "fromcurrent": True,
                "transition": {"duration": 500, "easing": "linear"}
            }],
            "label": "Play",
            "method": "animate"
        }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)
#fig.add_annotation(x = datetime(year=2024, month=5, day=26), y=100, text='DNF 26/05/2024', showarrow=True, arrowhead=1)
fig.add_layout_image(
    dict(
        source="https://th.bing.com/th?id=OIF.%2b90RqIvf%2fD8wBBpnHltrsg&rs=1&pid=ImgDetMain",  # URL ou chemin local de votre image
        xref="paper", yref="paper",
        x=0, y=0,  
        sizex=0.4, sizey=0.8,  
        xanchor="left", yanchor="bottom" 
    )
)
fig.update_layout(showlegend=False)
fig.show()
